Импорт библиотек

In [1]:
import zipfile
import json
import pandas as pd
import sqlite3

Инициализация констант

In [2]:
ZIP_ARCHIVE = 'egrul.json.zip'
DB_NAME = 'hw1.db'
TABLE_PART2 = 'telecom_companies'
DICT_KEY_SEARCH = 'СвОКВЭД'
OKVED_SEARCH = 'СвОКВЭДОсн'
SV_OKVED_KEY_SEARCH = 'КодОКВЭД'
SV_OKVED_VALUE_SEARCH = '61'


Инициализация переменных

In [ ]:
companys = pd.DataFrame()
count = 0

Чтение архива, и наполнение результирующего dataframe

In [3]:
with zipfile.ZipFile(ZIP_ARCHIVE, 'r') as z:
    for file_name in z.namelist():
        # if file_name == '00001.json':
        if file_name.endswith('.json'):
            count += 1
            print(f'Открытие файла {count} имя файла {file_name}')
            with z.open(file_name) as f:
                data = json.load(f)
                df = pd.DataFrame(data)
                df['svokved'] = df['data'].apply(lambda x: x[DICT_KEY_SEARCH] if DICT_KEY_SEARCH in x else {})
                df['svokvedosn'] = df['svokved'].apply(lambda x: x[OKVED_SEARCH] if OKVED_SEARCH in x else {})
                df['okved'] = df['svokvedosn'].apply(lambda x: x[SV_OKVED_KEY_SEARCH] if SV_OKVED_KEY_SEARCH in x and x[SV_OKVED_KEY_SEARCH][:2] == SV_OKVED_VALUE_SEARCH else None)
                df = df[~df['okved'].isna()][['ogrn','inn','kpp','name','full_name','okved']]
                companys = pd.concat([companys, df])


Открытие файла 1 имя файла 06439.json
Открытие файла 2 имя файла 04669.json
Открытие файла 3 имя файла 03534.json
Открытие файла 4 имя файла 00393.json
Открытие файла 5 имя файла 07388.json
Открытие файла 6 имя файла 10339.json
Открытие файла 7 имя файла 09162.json
Открытие файла 8 имя файла 05575.json
Открытие файла 9 имя файла 06849.json
Открытие файла 10 имя файла 08312.json
Открытие файла 11 имя файла 05296.json
Открытие файла 12 имя файла 08667.json
Открытие файла 13 имя файла 10549.json
Открытие файла 14 имя файла 10851.json
Открытие файла 15 имя файла 03216.json
Открытие файла 16 имя файла 07553.json
Открытие файла 17 имя файла 10245.json
Открытие файла 18 имя файла 08935.json
Открытие файла 19 имя файла 00981.json
Открытие файла 20 имя файла 07602.json
Открытие файла 21 имя файла 07293.json
Открытие файла 22 имя файла 08377.json
Открытие файла 23 имя файла 10193.json
Открытие файла 24 имя файла 08450.json
Открытие файла 25 имя файла 06832.json
Открытие файла 26 имя файла 03937.

Запись результата в БД

In [8]:
conn = sqlite3.connect(DB_NAME)
query = f'Create table if not Exists {TABLE_PART2} (ogrn text, inn text, kpp text, name text, full_name text, okved text)'
conn.execute(query)
companys.to_sql(TABLE_PART2,conn,if_exists='append',index=False)
conn.commit()
conn.close()